In [1]:
import numpy as np
import sys
sys.path.append('../general')
from pool import Pool
from metric import metric
from utils import predict_positions
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error
from collections import Counter

In [2]:
pool = Pool('../data')
pool.log_features()

In [3]:
def test_regression_model(model, pool):
    train_pool, test_pool = pool.train_test_split()
    train_pools = train_pool.split_by_position()

    model.fit(
        train_pool.regression_features, train_pool.targets,
        sample_weight=(1 / train_pool.probas)
    )

    score_predictions = model.predict(test_pool.regression_features)
    mse = mean_squared_error(score_predictions, test_pool.targets)

    positions = predict_positions(test_pool.features, model)
    positions_metric = metric(
        positions,
        test_pool.positions,
        test_pool.targets,
        test_pool.probas
    )
    
    return (
        mse,
        positions_metric,
        Counter(positions)
    )

In [4]:
results = [test_regression_model(LinearSVR(), pool) for i in range(10)]

In [8]:
metrics = [result[1] for result in results]
print(np.mean(metrics), np.std(metrics))

-0.0048894732915901965 0.012414712919272987


In [9]:
metrics

[-0.0018407633256934592,
 -0.03872227132714441,
 -0.0037293210539427663,
 0.002034540794302386,
 -0.00150910076121558,
 0.010859860359806437,
 -0.002059342277100942,
 -0.002009655015983852,
 -0.00048441846290852745,
 -0.01143426184602125]